In [ ]:
import pandas as pd
import numpy as np

np.random.seed(0)
n = 100
df = pd.read_csv('C:/Users/박서아/bootcamp_andrew_song/project/data/processed/tsla_processed.csv')

def cleaning_and_engineer_features(df):
    import numpy as np
    import pandas as pd
    
    # Create a copy to avoid modifying the original DataFrame
    df_eng = df.copy()
    
    # Replace empty strings with NaN and fill with appropriate values
    df_eng = df_eng.replace('', np.nan)
    
    # Handle missing values - use forward fill for time series data
    df_eng.fillna(method='ffill', inplace=True)
    df_eng.fillna(method='bfill', inplace=True)  # Backward fill any remaining NaNs
    
    # Recalculate technical indicators if they contain zeros or are missing
    if ('sma_10' in df_eng.columns and 
        (df_eng['sma_10'].eq(0).all() or df_eng['sma_10'].isnull().all())):
        print("Recalculating technical indicators...")
        df_eng['sma_10'] = df_eng['close'].rolling(window=10, min_periods=1).mean()
        df_eng['sma_20'] = df_eng['close'].rolling(window=20, min_periods=1).mean()
        df_eng['sma_50'] = df_eng['close'].rolling(window=50, min_periods=1).mean()
        df_eng['volatility_10'] = df_eng['close'].rolling(window=10, min_periods=1).std()
        df_eng['volatility_20'] = df_eng['close'].rolling(window=20, min_periods=1).std()
    
    # Recalculate ratios if SMAs exist
    if 'sma_10' in df_eng.columns:
        df_eng['price_sma_10_ratio'] = df_eng['close'] / df_eng['sma_10']
    if 'sma_20' in df_eng.columns:
        df_eng['price_sma_20_ratio'] = df_eng['close'] / df_eng['sma_20']
    
    # Add time-series specific features
    print("Adding time-series features...")
    for lag in [1, 2, 3, 5]:
        df_eng[f'close_lag_{lag}'] = df_eng['close'].shift(lag)
        df_eng[f'volume_lag_{lag}'] = df_eng['volume'].shift(lag)
    
    # Add return features
    df_eng['daily_return'] = df_eng['close'].pct_change(1)
    df_eng['return_5'] = df_eng['close'].pct_change(5)
    
    # Drop rows with NaN values after creating lag features
    df_eng = df_eng.dropna()
    
    return df_eng